In [1]:
# Solve the CPS Stata dataset questions (Q1–Q20)
# File path (already mounted in this environment):
DATA_PATH = "cps4_small.dta"

import numpy as np
import pandas as pd
from scipy.stats import skew
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import summary_table

# ----------------------------
# 1) Load data
# ----------------------------
df = pd.read_stata(DATA_PATH)

# Keep only relevant columns (but don't fail if extra columns exist)
needed = ["wage", "educ", "exper", "hrswk"]
missing = [c for c in needed if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns in dataset: {missing}")

# Drop rows with missing values in required columns
df = df[needed].dropna().copy()

# ----------------------------
# 2) Q1–Q4: Wage stats + skewness + log transform
# ----------------------------
wage_mean = df["wage"].mean()
wage_median = df["wage"].median()
wage_skew = skew(df["wage"], bias=False)

df["ln_wage"] = np.log(df["wage"])
lnw_skew = skew(df["ln_wage"], bias=False)

# ----------------------------
# 3) Regression 1:
# ln_wage = b0 + b1*educ + b2*exper + b3*hrswk + e
# ----------------------------
X1 = sm.add_constant(df[["educ", "exper", "hrswk"]])
m1 = sm.OLS(df["ln_wage"], X1).fit()

b1_educ = m1.params["educ"]
ci1_educ = m1.conf_int(alpha=0.05).loc["educ"].to_numpy()

b1_hr = m1.params["hrswk"]
ci1_hr_90 = m1.conf_int(alpha=0.10).loc["hrswk"].to_numpy()  # 90% CI

# significance check (alpha=0.05)
pvals1 = m1.pvalues

# Prediction for (educ=16, exper=10, hrswk=40)
x_pred1 = pd.DataFrame({"const": [1.0], "educ": [16.0], "exper": [10.0], "hrswk": [40.0]})
pred1 = m1.get_prediction(x_pred1).summary_frame(alpha=0.05)
# pred1 gives mean CI on ln scale; convert to wage scale by exponentiating
ln_hat = float(pred1["mean"])
ln_ci_lo = float(pred1["mean_ci_lower"])
ln_ci_hi = float(pred1["mean_ci_upper"])

w_hat = float(np.exp(ln_hat))
w_ci_lo = float(np.exp(ln_ci_lo))
w_ci_hi = float(np.exp(ln_ci_hi))

# ----------------------------
# 4) Regression 2:
# ln_wage = b0 + b1*educ + b2*exper + b3*hrswk + b4*exper^2 + e
# ----------------------------
df["exper2"] = df["exper"] ** 2
X2 = sm.add_constant(df[["educ", "exper", "hrswk", "exper2"]])
m2 = sm.OLS(df["ln_wage"], X2).fit()

# Marginal effect of experience on ln_wage: d ln_wage / d exper = b2 + 2*b4*exper
b2 = m2.params["exper"]
b4 = m2.params["exper2"]
cov = m2.cov_params()

def marginal_effect_exper(exper_value: float):
    """Returns (effect_on_ln, se_on_ln, 95% CI on ln scale) for d ln_wage / d exper."""
    x = exper_value
    effect = b2 + 2.0 * b4 * x

    # Delta method variance for linear combo: b2 + (2x)*b4
    # Var = Var(b2) + (2x)^2 Var(b4) + 2*(2x)*Cov(b2,b4)
    var = (
        cov.loc["exper", "exper"]
        + (2.0 * x) ** 2 * cov.loc["exper2", "exper2"]
        + 2.0 * (2.0 * x) * cov.loc["exper", "exper2"]
    )
    se = float(np.sqrt(var))

    z = 1.959963984540054  # ~ N(0,1) 97.5% quantile
    lo = float(effect - z * se)
    hi = float(effect + z * se)
    return float(effect), se, lo, hi

# Wendy: educ=18, exper=5 (marginal effect depends only on exper in this model)
wendy_eff, wendy_se, wendy_lo, wendy_hi = marginal_effect_exper(5.0)

# Jill: educ=18, exper=15
jill_eff, jill_se, jill_lo, jill_hi = marginal_effect_exper(15.0)

# Convert marginal effect on ln_wage to approximate percent effect on wage:
# For small changes, 100*(d ln_wage) ≈ % change in wage
wendy_pct = 100.0 * wendy_eff
wendy_pct_ci = (100.0 * wendy_lo, 100.0 * wendy_hi)

jill_pct = 100.0 * jill_eff
jill_pct_ci = (100.0 * jill_lo, 100.0 * jill_hi)

# ----------------------------
# 5) Map to multiple-choice options (A/B/C/D etc.)
#    (If none match exactly, we’ll print "no exact match")
# ----------------------------
def pick_option(value_tuple, options, tol=1e-2):
    """
    value_tuple can be a single float or (lo,hi) tuple.
    options: dict like {'a': (..), 'b': (..)}. Returns best match or None.
    tol: absolute tolerance for floats (or both endpoints).
    """
    best = None
    for k, v in options.items():
        if isinstance(value_tuple, tuple):
            if abs(value_tuple[0] - v[0]) <= tol and abs(value_tuple[1] - v[1]) <= tol:
                return k
        else:
            if abs(value_tuple - v) <= tol:
                return k
    return best

# Q1 options
q1_opts = {
    "a": (20.62, 17.3),
    "b": (12.83, 13.8),
    "c": (13.8, 12.83),
    "d": (20.62, 26.51),
}
q1_choice = pick_option((round(wage_mean, 2), round(wage_median, 2)), q1_opts, tol=0.02)

# Q2: skewness sign
q2_choice = "b" if wage_skew > 0 else ("a" if wage_skew < 0 else "c")

# Q3 is conceptual: correct is (a)
q3_choice = "a"

# Q4: ln_wage skewness closer to 0 than wage? (often “approximately normal”)
# We'll choose (c) if |skew| < 0.5; otherwise left/right by sign.
if abs(lnw_skew) < 0.5:
    q4_choice = "c"
else:
    q4_choice = "b" if lnw_skew > 0 else "a"

# Q5: interpret educ coefficient in log-linear model: approx 100*b1 percent
educ_pct = 100.0 * b1_educ
# options include 9.03%, 0.9%, etc. We'll match numerically.
q5_opts = {"a": 9.03, "b": 0.9}
q5_choice = pick_option(round(educ_pct, 2), q5_opts, tol=0.05) or "a/b? (see printed value)"

# Q6: 95% CI for percent increase from 1 more year educ
educ_ci_pct = (100.0 * ci1_educ[0], 100.0 * ci1_educ[1])
q6_opts = {
    "a": (0.078, 0.102),   # looks like decimals mistaken as %
    "c": (7.83, 10.22),
}
q6_choice = pick_option((round(educ_ci_pct[0], 2), round(educ_ci_pct[1], 2)), q6_opts, tol=0.08) or "c? (see printed CI)"

# Q7: % increase for +1 hour/week
hr_pct = 100.0 * b1_hr
q7_opts = {"a": 0.89, "b": 8.94, "c": 0.58, "d": 5.65}
q7_choice = pick_option(round(hr_pct, 2), q7_opts, tol=0.06) or "see printed value"

# Q8: 90% CI for +1 hour/week
hr_ci90_pct = (100.0 * ci1_hr_90[0], 100.0 * ci1_hr_90[1])
q8_opts = {
    "a": (0.58, 1.20),
    "b": (0.32, 0.83),
    "c": (0.63, 1.15),
    "d": (0.55, 1.25),
}
q8_choice = pick_option((round(hr_ci90_pct[0], 2), round(hr_ci90_pct[1], 2)), q8_opts, tol=0.08) or "see printed CI"

# Q9: which NOT significant at 0.05 in Regression 1
not_sig = [k for k in ["educ", "exper", "hrswk"] if pvals1[k] > 0.05]
if len(not_sig) == 0:
    q9_choice = "d"  # none of the above
elif not_sig == ["educ"]:
    q9_choice = "a"
elif not_sig == ["exper"]:
    q9_choice = "b"
elif not_sig == ["hrswk"]:
    q9_choice = "c"
else:
    q9_choice = f"multiple not sig: {not_sig}"

# Q10: estimated hourly wage at (16,10,40)
q10_opts = {"a": 19.314, "b": 2.961, "c": 2.603, "d": 9.031}
q10_choice = pick_option(round(w_hat, 3), q10_opts, tol=0.15) or "see printed value"

# Q11: 95% CI for estimated wage at (16,10,40)
q11_opts = {
    "a": (2.905, 3.017),
    "b": (18.263, 20.426),
    "c": (2.19, 2.90),
    "d": (15.34, 24.23),
}
q11_choice = pick_option((round(w_ci_lo, 3), round(w_ci_hi, 3)), q11_opts, tol=0.25) or "see printed CI"

# Q12: NOT significant at 0.05 in Regression 2 (educ, exper2, hrswk)
pvals2 = m2.pvalues
ns2 = []
if pvals2["educ"] > 0.05: ns2.append("a")
if pvals2["exper2"] > 0.05: ns2.append("b")
if pvals2["hrswk"] > 0.05: ns2.append("c")
q12_choice = "d" if len(ns2) == 0 else ", ".join(ns2)

# Q13 marginal effect expression: b2 + 2*b4*exper => option d
q13_choice = "d"

# Q14 theory: diminishing returns => b4 < 0 typically => option b if b4 negative
q14_choice = "b" if b4 < 0 else ("c" if b4 > 0 else "b/c? (b4≈0)")

# Q15 Wendy marginal effect (%)
q15_opts = {"a": 10.598, "b": 0.315, "c": 3.155, "d": 9.65}
q15_choice = pick_option(round(wendy_pct, 3), q15_opts, tol=0.25) or "see printed value"

# Q16 Wendy 95% CI
q16_opts = {
    "a": (1.529, 2.485),
    "b": (1.838, 3.927),
    "c": (2.376, 3.936),
    "d": (2.076, 4.636),
}
q16_choice = pick_option((round(wendy_pct_ci[0], 3), round(wendy_pct_ci[1], 3)), q16_opts, tol=0.30) or "see printed CI"

# Q17 Jill marginal effect (%)
q17_opts = {"a": 2.01, "b": 3.02, "c": 3.16, "d": 10.50}
q17_choice = pick_option(round(jill_pct, 2), q17_opts, tol=0.25) or "see printed value"

# Q18 Jill 95% CI
q18_choice = pick_option((round(jill_pct_ci[0], 3), round(jill_pct_ci[1], 3)), q16_opts, tol=0.30) or "see printed CI"

# Q19 compare marginal effects: if diminishing returns and b4<0 then Wendy(5) > Jill(15)
q19_choice = "b" if wendy_eff > jill_eff else ("a" if jill_eff > wendy_eff else "c")

# Q20 statement about diminishing returns if Wendy > Jill is True
q20_choice = True if wendy_eff > jill_eff else False

# ----------------------------
# 6) Print results neatly
# ----------------------------
print("=== Basic Wage Stats (Q1/Q2) ===")
print(f"Mean wage:   {wage_mean:.4f}")
print(f"Median wage: {wage_median:.4f}")
print(f"Skew(wage):  {wage_skew:.4f}  (positive => right-skew)")

print("\n=== Log Wage (Q4) ===")
print(f"Skew(ln_wage): {lnw_skew:.4f}")

print("\n=== Regression 1 Summary (ln_wage ~ educ + exper + hrswk) ===")
print(m1.summary())

print("\nKey quantities from Regression 1:")
print(f"educ coef: {b1_educ:.6f}  => ~{100*b1_educ:.3f}% per +1 year educ")
print(f"educ 95% CI (coef): [{ci1_educ[0]:.6f}, {ci1_educ[1]:.6f}]  => [{educ_ci_pct[0]:.3f}%, {educ_ci_pct[1]:.3f}%]")
print(f"hrswk coef: {b1_hr:.6f}  => ~{100*b1_hr:.3f}% per +1 hour/week")
print(f"hrswk 90% CI (coef): [{ci1_hr_90[0]:.6f}, {ci1_hr_90[1]:.6f}]  => [{hr_ci90_pct[0]:.3f}%, {hr_ci90_pct[1]:.3f}%]")

print("\nPrediction at educ=16, exper=10, hrswk=40 (Regression 1):")
print(f"Predicted ln_wage: {ln_hat:.6f}")
print(f"Predicted wage:    {w_hat:.4f}")
print(f"95% CI wage (mean): [{w_ci_lo:.4f}, {w_ci_hi:.4f}]")

print("\n=== Regression 2 Summary (ln_wage ~ educ + exper + hrswk + exper^2) ===")
print(m2.summary())

print("\nMarginal effect of experience (Regression 2): d ln_wage / d exper = b2 + 2*b4*exper")
print(f"b4 (exper2) = {b4:.8f}  (negative => diminishing returns)")

print("\nWendy (exper=5):")
print(f"Marginal effect on ln_wage: {wendy_eff:.6f}  => ~{wendy_pct:.3f}%")
print(f"95% CI: [{wendy_pct_ci[0]:.3f}%, {wendy_pct_ci[1]:.3f}%]")

print("\nJill (exper=15):")
print(f"Marginal effect on ln_wage: {jill_eff:.6f}  => ~{jill_pct:.3f}%")
print(f"95% CI: [{jill_pct_ci[0]:.3f}%, {jill_pct_ci[1]:.3f}%]")

print("\n=== Multiple-choice selections (best match) ===")
print(f"Q1: {q1_choice}   (mean={wage_mean:.2f}, median={wage_median:.2f})")
print(f"Q2: {q2_choice}   (skew={wage_skew:.3f})")
print(f"Q3: {q3_choice}")
print(f"Q4: {q4_choice}   (skew ln_wage={lnw_skew:.3f})")
print(f"Q5: {q5_choice}   (~{educ_pct:.2f}%)")
print(f"Q6: {q6_choice}   (~[{educ_ci_pct[0]:.2f}%, {educ_ci_pct[1]:.2f}%])")
print(f"Q7: {q7_choice}   (~{hr_pct:.2f}%)")
print(f"Q8: {q8_choice}   (~[{hr_ci90_pct[0]:.2f}%, {hr_ci90_pct[1]:.2f}%])")
print(f"Q9: {q9_choice}   (pvals: educ={pvals1['educ']:.4g}, exper={pvals1['exper']:.4g}, hrswk={pvals1['hrswk']:.4g})")
print(f"Q10:{q10_choice}  (w_hat={w_hat:.3f})")
print(f"Q11:{q11_choice}  (CI=[{w_ci_lo:.3f},{w_ci_hi:.3f}])")
print(f"Q12:{q12_choice}  (pvals2: educ={pvals2['educ']:.4g}, exper2={pvals2['exper2']:.4g}, hrswk={pvals2['hrswk']:.4g})")
print(f"Q13:{q13_choice}")
print(f"Q14:{q14_choice}  (b4={b4:.6g})")
print(f"Q15:{q15_choice}  (Wendy ~{wendy_pct:.3f}%)")
print(f"Q16:{q16_choice}  (Wendy CI ~[{wendy_pct_ci[0]:.3f}%, {wendy_pct_ci[1]:.3f}%])")
print(f"Q17:{q17_choice}  (Jill ~{jill_pct:.3f}%)")
print(f"Q18:{q18_choice}  (Jill CI ~[{jill_pct_ci[0]:.3f}%, {jill_pct_ci[1]:.3f}%])")
print(f"Q19:{q19_choice}  (Wendy eff={wendy_eff:.6f}, Jill eff={jill_eff:.6f})")
print(f"Q20:{q20_choice}")

=== Basic Wage Stats (Q1/Q2) ===
Mean wage:   20.6157
Median wage: 17.3000
Skew(wage):  1.5863  (positive => right-skew)

=== Log Wage (Q4) ===
Skew(ln_wage): 0.0503

=== Regression 1 Summary (ln_wage ~ educ + exper + hrswk) ===
                            OLS Regression Results                            
Dep. Variable:                ln_wage   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.217
Method:                 Least Squares   F-statistic:                     93.46
Date:                Tue, 24 Feb 2026   Prob (F-statistic):           2.69e-53
Time:                        21:02:15   Log-Likelihood:                -750.76
No. Observations:                1000   AIC:                             1510.
Df Residuals:                     996   BIC:                             1529.
Df Model:                           3                                         
Covariance Type:            nonrobust                       

C:\Users\26amr\AppData\Local\Temp\ipykernel_15644\4007256442.py:55: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ln_hat = float(pred1["mean"])
C:\Users\26amr\AppData\Local\Temp\ipykernel_15644\4007256442.py:56: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ln_ci_lo = float(pred1["mean_ci_lower"])
C:\Users\26amr\AppData\Local\Temp\ipykernel_15644\4007256442.py:57: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ln_ci_hi = float(pred1["mean_ci_upper"])
